<center><img src="https://github.com/FASSt-simulation/fasst_simulation_tools/raw/main/images/site-logo.png" width=150 height=150 alt="NGEE-Arctic Logo"/></center>

# Plot variables relevant to soil moisture and temperature from ELM output.

#### ----------------------------------------------------------------------------------------------------------------------

<br>

To run this script and generate the resulting ELM plots we need specific Python libraries.  This step loads those required libraries

In [ ]:
import matplotlib.pyplot as plt
import xarray
import os
import ipywidgets as widgets
import glob, numpy

Next, we locate the output data and identify which sites there is output for. This code creates a dropdown menu to pick the model run you want to plot.

In [ ]:
output_rootdir=os.path.expanduser('~')+'/output/cime_run_dirs/'
cases=sorted(glob.glob("%s*20TR*" % output_rootdir))
cases=[os.path.basename(x) for x in cases]
cases_dropdown = widgets.Dropdown(options=cases,
                                description='Choose Case Name:',
                                style={'description_width':'auto'},
                                layout={'width':'max-content'},
                                disabled=False)

display(cases_dropdown)

Here, we are using the combined output file over all years of the run. We read in the list of variables, units, and longer descriptions from the dataset. We use the time variable from the dataset to determine the start and end years.

In [ ]:
output_file=output_rootdir + cases_dropdown.value + '/run/ELM_output.nc'

# Read in the list of variable, units, and longer descriptions from the dataset
with xarray.open_dataset(output_file) as data:
    # Determine start and end year of the dataset
    data_startyear=data.time[0].item().year
    data_endyear=data.time[-1].item().year

startyear_picker = widgets.BoundedIntText(value=data_startyear, min=data_startyear, max=data_endyear, step=1,
                                          description='Choose start year',style={'description_width':'auto'})
endyear_picker = widgets.BoundedIntText(value=data_endyear, min=data_startyear, max=data_endyear, step=1,
                                          description='Choose end year',style={'description_width':'auto'})

# Use smoothing if we add surface energy balance?
# smoothing_picker = widgets.IntText(value=0,description='Choose smoothing on fluxes (months)',style={'description_width':'auto'})

display(startyear_picker)
display(endyear_picker)
# display(smoothing_picker)

Using the fields above, you can pick the start and end year of the time series to visualize.

Next, we read in the dataset and select the time portion based on start and end year picked above.

In [ ]:
year_start=startyear_picker.value
year_end=endyear_picker.value
# smoothing=smoothing_picker.value

elm_output=xarray.open_dataset(output_file).squeeze().sel(time=slice(str(year_start),str(year_end)))

Now we set up a figure and plot the relevant variables in different panels.

In [ ]:
# Soil temperature and freezing

# Set up a figure with three rows
fig,a=plt.subplots(nrows=4,clear=True,num='Soil temperature and ice',figsize=(15,13))

# Subset output to the 1950-2010 period

ax=a[0]
elm_output['ALTMAX'].plot(ax=ax,label='Maximum active layer thickness',color='C1')
elm_output['ZWT'].plot(ax=ax,label='Water table depth',color='blue')
elm_output['ZWT_PERCH'].plot(ax=ax,label='Perched water table depth',color='blue',linestyle='--')
ax.set(title='Active layer and water table',xlabel='',ylabel='Depth (m)')
ax.legend()

ax=a[1]
elm_output['SNOW_DEPTH'].plot(ax=ax,label='Snow depth')
ax.set(title='Snow depth',xlabel='',ylabel='Depth (m)')

ax=a[2]
(elm_output['TSOI']-273.15).T.plot(ax=ax,cbar_kwargs={'label':'Soil temperature (C)'},center=0,vmin=-10,vmax=10,cmap='coolwarm')
elm_output['ALTMAX'].plot(ax=ax,label='Maximum active layer thickness',color='k',linestyle=':',linewidth=0.5)
ax.set(title='Soil temperature',xlabel='',ylabel='Depth (m)',ylim=(20,0))

ax=a[3]
(elm_output['SOILICE']/(elm_output['SOILLIQ']+elm_output['SOILICE'])).T.plot(ax=ax,cbar_kwargs={'label':'Soil frozen water fraction'},cmap='Blues_r')
ax.set(title='Soil frozen water',xlabel='Year',ylabel='Depth (m)',ylim=(3,0))

<center><img src="https://github.com/FASSt-simulation/fasst_simulation_tools/raw/main/images/site-logo.png" width=150 height=150 alt="NGEE-Arctic Logo"/></center>